## For Assignment 1

### https://www.kaggle.com/uciml/pima-indians-diabetes-database - Download the indian diabetes dataset

### 1.Import relevant commands for numpy, pandas, sklearn.

In [2]:
import numpy as np
import scipy.stats as stats
import math
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
FOLDS=10
ALPHA=0.9

### 2.Using the appropriate pandas function, read the diabetes.csv into a dataframe. Pay good attention to the necessary arguments.

In [3]:
data = pd.read_csv('diabetes.csv', sep=",")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### 3.use naivebayes, logistic regression and 3-nn classifiers (library) to train on the training sets and compute training and validation errors for each fold. The target label is Outcome.

1. Split the dataset into training and testing set and apply all operations on training set.
1. Create 10 folds on training set.
2. At each fold, train all classifiers and calculate their training and validation error at each fold.

In [25]:
features=data.iloc[:,:-1].values
labels=data.iloc[:,-1].values
x_train,x_test,y_train,y_test=train_test_split(features,labels,train_size=0.8)

knn_classifier=KNeighborsClassifier(n_neighbors=3)
nb_classifier=GaussianNB()
log_reg_classifier=LogisticRegression(solver='liblinear')

kfold=KFold(n_splits=FOLDS,shuffle=True)

knn_validation_errors=[]
nb_validation_errors=[]
log_reg_validation_errors=[]

knn_training_errors=[]
nb_training_errors=[]
log_reg_training_errors=[]

for train_indices,validation_indices in kfold.split(y_train):
    nb_classifier.fit(x_train[train_indices],y_train[train_indices])
    training_predictions=nb_classifier.predict(x_train[train_indices])
    nb_training_errors.append(np.sum(training_predictions!=y_train[train_indices])/len(training_predictions))
    predictions=nb_classifier.predict(x_train[validation_indices])
    nb_validation_errors.append(np.sum(predictions!=y_train[validation_indices])/len(predictions))
    
    knn_classifier.fit(x_train[train_indices],y_train[train_indices])
    training_predictions=knn_classifier.predict(x_train[train_indices])
    knn_training_errors.append(np.sum(training_predictions!=y_train[train_indices])/len(training_predictions))
    predictions=knn_classifier.predict(x_train[validation_indices])
    knn_validation_errors.append(np.sum(predictions!=y_train[validation_indices])/len(predictions))
    
    log_reg_classifier.fit(x_train[train_indices],y_train[train_indices])
    training_predictions=log_reg_classifier.predict(x_train[train_indices])
    log_reg_training_errors.append(np.sum(training_predictions!=y_train[train_indices])/len(training_predictions))
    predictions=log_reg_classifier.predict(x_train[validation_indices])
    log_reg_validation_errors.append(np.sum(predictions!=y_train[validation_indices])/len(predictions))

    
knn_validation_errors=np.array(knn_validation_errors)
nb_validation_errors=np.array(nb_validation_errors)
log_reg_validation_errors=np.array(log_reg_validation_errors)

training_errors=pd.DataFrame({"3-NN":knn_training_errors,"Logistic Regression":log_reg_training_errors,"Naive Bayes":nb_training_errors},index=["Fold "+str(i+1) for i in range(FOLDS)])
validation_errors=pd.DataFrame({"3-NN":knn_training_errors,"Logistic Regression":log_reg_validation_errors,"Naive Bayes":nb_validation_errors},index=["Fold "+str(i+1) for i in range(FOLDS)])

print("Training Errors:")
print(training_errors)
print("\n\nValidation Errors:")
print(validation_errors)

Training Errors:
             3-NN  Logistic Regression  Naive Bayes
Fold 1   0.153986             0.215580     0.235507
Fold 2   0.159420             0.217391     0.240942
Fold 3   0.163043             0.217391     0.244565
Fold 4   0.164855             0.233696     0.240942
Fold 5   0.168174             0.218807     0.238698
Fold 6   0.164557             0.224231     0.244123
Fold 7   0.146474             0.213382     0.236890
Fold 8   0.162749             0.216998     0.244123
Fold 9   0.166365             0.224231     0.238698
Fold 10  0.164557             0.220615     0.247740


Validation Errors:
             3-NN  Logistic Regression  Naive Bayes
Fold 1   0.153986             0.258065     0.274194
Fold 2   0.159420             0.193548     0.290323
Fold 3   0.163043             0.193548     0.225806
Fold 4   0.164855             0.209677     0.225806
Fold 5   0.168174             0.245902     0.311475
Fold 6   0.164557             0.196721     0.180328
Fold 7   0.146474         

### 4.is the error of naive bayes <0.2 with confidence 0.9

1. We need to perform **t-test** to check if the error is less than 0.2
2. Confidence level=0.9
3. p0=0.2
4. Calculate t-value and compare it with t-critical 

In [12]:
mean=np.mean(nb_validation_errors)
std_dev=np.std(nb_validation_errors)
t_k_1=np.sqrt(FOLDS)*(mean-0.2)/std_dev
df=FOLDS-1
t=stats.t.ppf(ALPHA,df)

print("T value:",t_k_1)
print("T critical:",t)
if t_k_1>t:
    print("No")
else:
    print("Yes")


T value: 3.323858116430678
T critical: 1.3830287383964925
No


### 5.have  naive bayes and knn the same error?
1. We need to perform **K-Fold Cross Validation Paired Test** to check whether 2 classifiers have same errors.
2. Confidence level= 0.95 (It is not mentioned in the question. So, I assumed confidence level is 0.95)
3. Calculate paired differences for 10 folds.
4. Calculate t-value and perform 2 tailed test.

In [21]:
paired_difference=nb_validation_errors-knn_validation_errors
mean=np.mean(paired_difference)
std_dev=np.std(paired_difference)
t_k_1=np.sqrt(FOLDS)*mean/std_dev
t_critical=stats.t.ppf(0.95/2,df)
print("T value:",t_k_1)
print("T critical:",t_critical)
if t_k_1>=t_critical and t_k_1<=-t_critical:
    print("They have same error")
else:
    print("They have different errors")

T value: -2.522293485296181
T critical: -0.06447679010158362
They have different errors


### 6.do the three classifiers have different errors?
1. We need to perform **ANOVA** to check if all classifiers have different errors.
2. Confidence level= 0.95 (It is not mentioned in the question. So, I assumed confidence level is 0.95)
3. Calculate first estimator with between groups sum of squares.
4. Calculate second estimator with within group sum of squares.
5. Take ratio of both estimators and check if this value is greater than F-value at given confidence level.

In [24]:
error_table=np.array(np.transpose([knn_validation_errors,nb_validation_errors,log_reg_validation_errors]))
no_of_models=np.shape(error_table)[1]
classifier_error_average=np.mean(error_table,axis=0)
estimator_b=FOLDS*np.var(classifier_error_average)
estimator_w=np.sum(np.var(error_table,axis=0)/no_of_models)
f_value=stats.f.ppf(0.95,dfn=no_of_models-1,dfd=no_of_models*(FOLDS-1))
print("F-value:",f_value)
print("Estimators ratio:",estimator_b/estimator_w)
if estimator_b/estimator_w<f_value:
    print("All classifiers have same errors")
else:
    print("All classifiers have different errors")

F-value: 3.3541308285291986
Estimator ratio: 4.588719236495855
All classifiers have different errors


### 7.Use Bayes rule to decide on the label using the Glucose feature. Compute the mean and std of Glucose feature on the whole dataset (marginal distribution) and for each class separately, (class condition distribution Prob(x|C=0), Prob(x|C=1))). Assume that the feature is distributed according to the mean and std you computed. Compute the predictions for the 10 validation sets. Compare with the naivebayes classifier  (library) using only the Glucose feature

1. Create NaiveBayes class with fit and predict methods.
2. In training, calculate mean and variance of every class.
3. Assumption:- Distribution is Gaussian.
4. For calculation of conditional probability use gaussian function.
5. For prediction, calculate P(C)P(X|C).
6. If P(0)P(X|0)>P(1)P(X|1) yhen predict class 0 else predict class 1.

In [82]:
class NaiveBayes:
    def __init__(self):
        self.mean=[]
        self.variance=[]
        self.prior=[]
    def conditional_probability(self,x):
        result=[]
        for i in range(self.mean.shape[0]):
            numerator=np.exp(-((x-self.mean[i])**2)/(2*self.variance[i]))
            denominator=np.sqrt(2*np.pi*self.variance[i])
            result.append((numerator/denominator).item())
        return np.array(result)
    def fit(self,x_train,y_train):
        no_of_classes=np.unique(y_train).shape[0]
        for i in range(no_of_classes):
            raw_data=x_train[y_train==i]
            self.prior.append(raw_data.shape[0]/x_train.shape[0])
            self.mean.append(np.mean(raw_data))
            self.variance.append(np.var(raw_data))
        self.mean=np.array(self.mean)
        self.variance=np.array(self.variance)
        self.prior=np.array(self.prior)
    def predict(self,x):
        predictions=[]
        for x_data in x:
            probabilities=self.conditional_probability(x_data)*self.prior
            predictions.append(np.argmax(probabilities))
        return np.array(predictions)

In [83]:
x=data.loc[:,["Glucose"]].values
y=data.iloc[:,-1].values

sklearn_error=[]
custom_nb_error=[]


for train,validation in kfold.split(y_train):
    custom_model=NaiveBayes()
    sklearn_model=GaussianNB()
    
    custom_model.fit(x[train],y[train])
    predictions=custom_model.predict(x[validation])
    error=np.sum(predictions!=y[validation])/np.shape(y[validation])[0]
    custom_nb_error.append(error)
    
    sklearn_model.fit(x[train],y[train])
    predictions=sklearn_model.predict(x[validation])
    error=np.sum(predictions!=y[validation])/np.shape(y[validation])[0]
    sklearn_error.append(error)

results=pd.DataFrame({"Library model":sklearn_error,"Custom model":custom_nb_error})
print(results)

   Library model  Custom model
0       0.274194      0.274194
1       0.161290      0.161290
2       0.274194      0.274194
3       0.258065      0.258065
4       0.213115      0.213115
5       0.262295      0.262295
6       0.295082      0.295082
7       0.229508      0.229508
8       0.327869      0.327869
9       0.278689      0.278689
